In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sys

In [ ]:
# import reusable functions from utils directory
sys.path.append('../../utils')
import functions

# **Cleaning Data Conclusions**

## Day 2
- `merged_final_demo_final_experiment_clients_df`

# Final Demo + Final Experiment Clients (Merged DataFrame)

In [ ]:
final_demo_df = pd.read_csv('../../data/clean/merged_final_demo_final_experiment_clients_df.csv')
final_demo_df.head()

In [ ]:
# functions.inspect_dataframe(final_demo_df)

In [ ]:
functions.check_unique_and_empty(final_demo_df)

### Table Overview

**Rows:**

- `client_id (int)`: A unique identifier for each client, used to distinguish one client from another in the dataset.
- `client_tenure_years (int)`: The number of years a client has been associated with the company. For example, a client with client_tenure_years = 6 has been with the company for 6 years.
- `client_tenure_months (int)`: The number of months a client has been associated with the company. This value is often more granular than client_tenure_years and could be used for more detailed analysis. For instance, a tenure of 6 years and 1 month would be represented as 73 months.
- `client_age (int)`: The age of the client in years.
- `gender`: The gender of the client. The value can be "Male," "Female," or "Unspecified," meaning the gender data is either recorded or missing.
- `num_accounts (int)`: The number of accounts the client has with the company.
- `balance (float)`: The total balance of the client's accounts with the company. This is a monetary value, and the balance can indicate how much money the client holds across their accounts.
- `calls_last_6_months (int)`: The number of calls the client has made to the company in the past six months. This can give an idea of how actively the client has engaged with the company.
- `logons_last_6_months (int)`: The number of times the client has logged into their account or interacted with the company online in the past six months.
- `variation (object)`: This column likely indicates whether the client is part of a control group or a test group for an experiment. In this case, clients are either labeled as "Test", "Control", or "Unknown".

## Day 1 & 2 (Week 5)

### **Client behavior analysis**

Answer the following questions about demographics:

- Who are the primary clients using this online process?
- Are the primary clients younger or older, new or long-standing?
- Next, carry out a client behaviour analysis to answer any additional relevant questions you think are important.

In [ ]:
# Step 1: Analyze Active Clients using the online process

# assuming active clients are those with more than 10 logons in the last 6 months and more than 3 accounts
active_clients = final_demo_df[(final_demo_df['logons_last_6_months'] > 10) & (final_demo_df['accounts'] > 3)]

# Step 2: Are the primary clients younger or older?

average_age = final_demo_df['age'].mean()

# age distribution to visualize client ages
plt.figure(figsize=(8, 6))
sns.histplot(final_demo_df['age'], kde=True, color="blue", bins=10)
plt.title('Client Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

# Step 3: Are the primary clients newer or long-standing?
# Calculate the average tenure (in years)
average_tenure = final_demo_df['tenure_years'].mean()

# Plot the tenure distribution to visualize how long clients have been with the service
plt.figure(figsize=(8, 6))
sns.histplot(final_demo_df['tenure_years'], kde=True, color='green', bins=10)
plt.title('Client Tenure Distribution (in years)')
plt.xlabel('Year duration')
plt.ylabel('Frequency')
plt.show()

# Step 4: Drawing Conclusions

# 1. Active clients using the online process
print('Active clients (higher logons and accounts):')
print(active_clients)

# 2. Average age of clients
print(f'Average client age: {average_age:.2f} years')

# 3. Average client tenure (years)
print(f'Average client tenure: {average_tenure:.2f} years')

# conclusions:
if average_age < 40:
    print('Conclusion: The primary clients are generally younger (under 40 years old).')
else:
    print('Conclusion: The primary clients are generally older (above 40 years old).')

if average_tenure < 3:
    print('Conclusion: The primary clients are generally newer (under 3 years).')
else:
    print('Conclusion: The primary clients are generally long-standing (over 3 years).')

## Day 3 (Week 5)

### **Performance Metrics**

**Success Indicators**

Discovering what key performance indicators (KPIs) will determine the success of the new design.
Use at least completion rate, time spent on each step and error rates. Add any KPIs you might find relevant.

## **KPIs - Completion Rate**
The proportion of users who reach the final `confirm` step.

**How**

- Analyzing the dataset specifically focusing on calculating the completion rate, based on the steps that clients take. We initially convert the `step` names to numerical values for easier calculation. Our code filters the rows where `step` is equal to 4 (the final step or completion of the process). `clients_finished` now contains all records of clients who completed the process.

- Using `.nunique()` we calculate the total numbers of distinct `client_id` values in the dataframe, getting the count of unique clients. With that, we calculate the number of unique clients who finished the process.

- Grouping `clients_finished` by `client_id` we count how many times each flient has completed the process. All results are stored in a new dataframe called `completion_count_df`, where `completion_count` represents the number of times each client reached step 4.

- Any `NaN` values in the `completion_count` column will be replaced by 0, indicating that these clients never completed the process.

- These new columns and data will be added to the dataframe.

### **Results**

The result *"Clients who finished the process: 47,787 out of 70,594"* provides a summary of how many clients in the dataset have successfully completed the process (reached the final step, which is step 4 in this case), compared to the total number of clients.

#### **Breaking Down the Information:**

- **Clients who finished the process (47,787):**
This number represents the clients who reached the final step (step 4) in the project or process. It's a count of how many clients successfully completed the entire journey or task.

- **Total unique clients (70,594):**
This is the total number of unique clients in the dataset. The client_id column likely represents individual clients, and nunique() is used to count how many distinct clients are in the dataset, regardless of how many steps they completed.

#### **What Can We Infer?**

The ratio of clients who finished the process to the total number of clients can be calculated as:

**Completion Rate = 47,78770,594 × 100 ≈ 67.7%**

**Completion Rate = 70,59447,787​ × 100 ≈ 67.7%**

So, about 67.7% of the clients who started the process completed it.

- **Client Drop-off:**
The remaining 32.3% of clients (i.e., 70,594 - 47,787 = 22,807) did not reach the final step. This could indicate a drop-off or abandonment rate, where clients started but didn’t finish.
Understanding the reasons for this drop-off (e.g., user experience issues, complexity, lack of incentives) could help improve the process or identify areas for intervention to increase completion rates.

- **Implications for Project Metrics:**
A 67.7% completion rate can be seen as relatively good in many contexts, especially if the process is long or involves several steps. However, in some industries or projects, you may want to aim for a higher completion rate.
If this is a metric for performance or KPIs (Key Performance Indicators), you might want to set a target (e.g., 80% completion rate) and use this data to track progress toward that goal.

#### **Potential Follow-up Analysis:**

- **Examine the Drop-off Points:**
You might want to analyze where the drop-offs occur in the steps leading up to step 4. For instance, do most clients drop off at a specific step (e.g., step 3), or is the drop-off more evenly distributed?
Investigating this can give insight into potential bottlenecks or friction points in the process.

- **Segment Clients:**
Segmenting clients by other characteristics (e.g., demographics, usage patterns, source of acquisition) could reveal if some groups are more likely to complete the process than others. This could help tailor interventions for high-value or at-risk clients.

- **Multiple Completions:**
If some clients complete the process multiple times (as might be possible in certain scenarios), this could skew the results. You could consider focusing on unique clients who completed the process at least once versus total completions.

- **Client Retention:**
If the process completion rate is tied to retention or customer loyalty (e.g., users who finish the process are more likely to stay), then this result could be a strong indicator of overall client engagement.

#### **Conclusion:**

The result shows that **approximately 67.7% of clients successfully completed the process, while about 32.3% did not**. This completion rate could be a useful KPI for understanding how effectively the project is engaging and retaining clients, as well as identifying areas for improvement in the process.


![Image Description](../../visualizations/kpi_plots/barchart_total_vs_completed_clients.png)


![Image Description](../../visualizations/kpi_plots/piechart_completion_rate_clients.png)


## **KPIs - Time Spent on Each Step**
The average duration users spend on each step.

**How:**

Our code calculates the average time spent by clients on each step of a process during each visit, using the timestamp data for each step. The key idea is to calculate the time difference between consecutive steps for each client and then group this data by `client_id` and `step` to compute average times. The final results are presented in both seconds and minutes.

- We ensured that the `date_time` column is in a proper datetime format so it can be computed.
- The dataframe is sorted by `client_id`, `visit_id`, and `step` to ensure chronological order. This is mandatory for calculating the time differences correctly.
- We created a `time_spent` column to substract the current step's `date_time` from the next `step_time`, this gives the duration spent on the current step.
- `shift(-1) is used to shift the `date_time` values for each group, effectively getting the timestamp of the next step in sequence.
- For step 4, there is no "next step" because it's the final step in the process. The code calculates the `next_step_time` as the `maximum date_time` for each visit (`visit_id`), assuming this represents the end of the session.
- After updating the `next_step_time` for step 4, the `time_spent` for step 4 is recalculated.
- The data is grouped by `client_id` and `step`, and the mean time spent on each step is calculated for each client. This gives the average time spent per step for each client. The average time spent will also be calculated in minutes and seconds for further reading.


The final dataframe, `avg_time_per_step`, contains the following columns:
- `client_id`: The unique client identifier.
- `step`: The step number (1, 2, 3, or 4).
- `avg_time_spent`: The average time spent on each step (as a timedelta object).
- `avg_time_seconds`: The average time spent in seconds.
- `avg_time_minutes`: The average time spent in minutes.


The function returns a dataframe `avg_time_per_step`, which contains the average time spent by each client on each step of the process. This is useful for understanding how much time clients typically spend at each stage of the process, which can help identify bottlenecks or stages that need improvement.

### **Results**

| Step      | Average time (minutes) |
|-----------|--------------------------|
| Start     | 0.673287                 |
| Step 1    | 0.813043                 |
| Step 2    | 1.563080                 |
| Step 3    | 2.144761                 |
| Finish    | 0.320263                 |

#### **Conclusion:**

The average time analysis indicates that while Steps 2 and 3 may require additional client attention or effort, Step 4 shows an unexpected decrease in time spent. Understanding why clients spend less time on Step 4 could reveal potential areas for improvement in user experience or process completion rates. Optimizing Step 2 and Step 3, which are the longest steps, might also enhance client engagement and process efficiency.


Notice that what is considered a step to calculate is a very subjective decision.

![Image Description](../../visualizations/kpi_plots/barplot_avg_time_per_step.png)

![Image Description](../../visualizations/kpi_plots/lineplot_avg_time_per_step.png)


## **KPIs - Error Rates**
If there’s a step where users go back to a previous step, it may indicate confusion or an error. You should consider moving from a later step to an earlier one as an error.

**How**

### **Results**


#### **Breaking Down the Information:**

#### **What Can We Infer?**

#### **Potential Follow-up Analysis:**

#### **Conclusion:**
